# Questão 1: Classificação Supervisionada de Sinais em Libras

O objetivo desta questão é desenvolver e avaliar modelos de Machine Learning capazes de classificar 25 sinais distintos da Língua Brasileira de Sinais (Libras). O ponto de partida é um corpus de dados brutos composto por arquivos JSON, onde cada arquivo contém as coordenadas dos keypoints (pontos-chave) do corpo de um intérprete ao longo de uma sequência de frames.

Para que os algoritmos de classificação possam aprender os padrões dos movimentos, foi necessário realizar um robusto processo de pré-processamento e engenharia de features.
As etapas a seguir descrevem a transformação dos dados brutos em um formato estruturado e rico em informações.



### 1.1 Pré-processamento e Consolidação dos Dados

A primeira etapa consistiu em consolidar os dados de todos os arquivos JSON em uma única estrutura.

* **Criação do DataFrame Longo:** Utilizando o script **`limpeza_json2.py`**, os múltiplos arquivos da pasta `Sinais` foram lidos e processados. Essa ação transformou os dados brutos em um único DataFrame no formato "longo" (`df_sinal`), que foi salvo como **`sinais_long.csv`**. Neste arquivo, cada linha representa um único keypoint (`id`) em um único `frame` de um vídeo.

* **Enriquecimento para Normalização:** Para viabilizar a normalização das coordenadas, o DataFrame `sinais_long.csv` foi mesclado com o arquivo de metadados `sinais.csv`. O objetivo principal desta junção foi trazer as colunas **`width` e `height`** para cada linha, associando as dimensões de cada vídeo aos seus respectivos keypoints.

* **Normalização das Coordenadas:** As coordenadas `x` e `y` originais estavam em pixels. Para criar um padrão consistente, elas foram normalizadas para uma escala de 0 a 1, dividindo `x` pela largura (`width`) e `y` pela altura (`height`) do vídeo correspondente. Isso garante que os modelos analisem o padrão do movimento, independentemente do tamanho do vídeo.

* **Filtragem de Keypoints:** Com base na natureza da Libras, os movimentos mais relevantes para a distinção dos sinais estão concentrados nos braços e mãos. Portanto, o DataFrame foi filtrado para manter apenas os keypoints de ID 11 a 22, removendo os dados do rosto, quadril e pernas, o que ajuda a reduzir o ruído para o modelo.



### 1.2 Engenharia de Features Temporais

Com os dados limpos e normalizados, o passo seguinte foi a engenharia de features, projetada para transformar a sequência de movimentos em um vetor de características de tamanho fixo para cada vídeo.

A estratégia adotada foi a seguinte:

1.  **Segmentação Temporal:** A sequência de frames de cada vídeo foi dividida em **10 segmentos (partes) proporcionais** à sua duração. Isso nos permite analisar a evolução do sinal em 10 estágios, do início ao fim.

2.  **Extração de Características Estatísticas:** Para cada um dos 10 segmentos, foram calculadas três métricas estatísticas (`média`, `desvio padrão` e `variância`) para as coordenadas `x` e `y` de cada um dos 12 keypoints.
    * A **média** nos informa a posição central da articulação.
    * O **desvio padrão** e a **variância** nos dão uma ideia da amplitude e da dinâmica do movimento naquele estágio.

Este processo resultou em um DataFrame final (`data_merged1.csv`) com **720 features** para cada vídeo (`10 partes * 12 keypoints * 2 coordenadas * 3 métricas`), onde cada linha representa um sinal completo (cada linha representada por um video(json)), pronto para ser utilizado nos algoritmos de classificação.


In [1]:
#Bibliotecas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from unidecode import unidecode

In [2]:
#aumentado a capacidade de visualização de colunas e linhas
#import pandas as pd

#pd.set_option('display.max_columns', 7000)
#pd.set_option('display.max_rows',90000)

### Análise e Limpeza Inicial do DataFrame de Metadados

O primeiro passo consiste em carregar o arquivo de metadados `sinais.csv`, que contém informações essenciais sobre cada vídeo, como o nome do arquivo, a duração, o sinal correspondente e o intérprete. Este DataFrame será referenciado pela variável `data`.

Após uma análise inicial, foi identificada uma linha que não possuía a informação do `file_name`, tornando-a impossível de ser associada aos dados de keypoints. Para garantir a integridade do conjunto de dados, essa linha foi removida.

In [3]:
# Transformação do sinais.csv em um Dataframe
data = pd.read_csv("sinais.csv")
data.head(11)

,file_name,width,height,duration_sec,num_frames,sinal,interprete
0,Adição_AP_10.json,738,1008,4.533333,136,Adição,Alexson
1,Adição_AP_1.json,774,1006,4.766667,143,Adição,Alexson
2,Adição_AP_2.json,760,1002,4.433333,133,Adição,Alexson
3,Adição_AP_3.json,762,1000,4.933333,148,Adição,Alexson
4,Adição_AP_4.json,764,1004,4.600000,138,Adição,Alexson
5,Adição_AP_5.json,762,1002,4.366667,131,Adição,Alexson
6,Adição_AP_6.json,762,1014,5.233333,157,Adição,Alexson
7,Adição_AP_7.json,772,998,4.933333,148,Adição,Alexson
8,Adição_AP_8.json,774,1004,4.800000,144,Adição,Alexson
9,Adição_AP_9.json,772,1008,5.833333,175,Adição,Alexson


In [4]:
# Verificando a quantidade de interpretes 
# Quantidade de intérpretes diferentes
num_interpretes = data["interprete"].nunique()
print("Número de intérpretes únicos:", num_interpretes)

# Lista dos intérpretes
lista_interpretes = data["interprete"].unique()
print("Lista de intérpretes:", lista_interpretes)

# Quantidade de amostras por intérprete
contagem = data["interprete"].value_counts()
print("\nAmostras por intérprete:\n", contagem)


Número de intérpretes únicos: 10
Lista de intérpretes: ['Alexson' 'Aparecida' 'Cecilia' 'Dannubia' 'Dilainne' 'Everton'
 'Jackeline' 'Luana' 'Rosenice' 'Tiago']

Amostras por intérprete:
 interprete
Alexson      251
Cecilia      251
Aparecida    250
Dannubia     250
Dilainne     250
Everton      250
Jackeline    250
Luana        250
Rosenice     250
Tiago        250
Name: count, dtype: int64


In [5]:
# Informações sobre o dataframe
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2502 entries, 0 to 2501
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   file_name     2501 non-null   object 
 1   width         2502 non-null   int64  
 2   height        2502 non-null   int64  
 3   duration_sec  2502 non-null   float64
 4   num_frames    2502 non-null   int64  
 5   sinal         2502 non-null   object 
 6   interprete    2502 non-null   object 
dtypes: float64(1), int64(3), object(3)
memory usage: 137.0+ KB


In [6]:
data.shape

(2502, 7)

In [7]:
# Remove linhas com file_name vazio ou NaN
#Percebi que existe um file_name vazio
data = data[~(data['file_name'].isna() | (data['file_name'] == ''))]

In [8]:
# Verificando o shape depois da remoção da linha
data.shape

(2501, 7)

In [9]:
#Criação do DataFrame json longo
df_json_longo = pd.read_csv("sinais_long.csv")

In [10]:
df_json_longo.head()

,file_name,frame,id,x,y,z,visibility
0,Adição_AP_1.json,0,0,438,184,-1.44,1.0
1,Adição_AP_1.json,0,1,462,154,-1.36,1.0
2,Adição_AP_1.json,0,2,474,154,-1.36,1.0
3,Adição_AP_1.json,0,3,484,154,-1.36,1.0
4,Adição_AP_1.json,0,4,421,155,-1.36,1.0


In [11]:
# Merge do df_json_longo com as colunas width e height do data
# Merge dos dataframes para adicionar width e height
df_final_longo = df_json_longo.merge(
    data[['file_name', 'width', 'height']],  # pega só colunas relevantes
    on='file_name',
    how='left'  # mantém todas as linhas do json
)
# Reorganizar as colunas
df_final_longo = df_final_longo[
    ["file_name", "frame", "id", "x", "y", "width", "height", "z", "visibility"]
]


In [12]:
#Variavel do dataframe json longo com width e height de sinais.csv
df_final_longo.head()

,file_name,frame,id,x,y,width,height,z,visibility
0,Adição_AP_1.json,0,0,438,184,774,1006,-1.44,1.0
1,Adição_AP_1.json,0,1,462,154,774,1006,-1.36,1.0
2,Adição_AP_1.json,0,2,474,154,774,1006,-1.36,1.0
3,Adição_AP_1.json,0,3,484,154,774,1006,-1.36,1.0
4,Adição_AP_1.json,0,4,421,155,774,1006,-1.36,1.0


In [13]:
# Normalizar x e y diretamente
df_final_longo['x'] = df_final_longo['x'] / df_final_longo['width']
df_final_longo['y'] = df_final_longo['y'] / df_final_longo['height']


In [14]:
# Clip para garantir valores no intervalo [0, 1]
df_final_longo['x'] = df_final_longo['x'].clip(0, 1)
df_final_longo['y'] = df_final_longo['y'].clip(0, 1)

In [15]:
# A normlização foi feita corretamente
df_final_longo.head(25)

,file_name,frame,id,x,y,width,height,z,visibility
0,Adição_AP_1.json,0,0,0.565891,0.182903,774,1006,-1.44,1.00
1,Adição_AP_1.json,0,1,0.596899,0.153082,774,1006,-1.36,1.00
2,Adição_AP_1.json,0,2,0.612403,0.153082,774,1006,-1.36,1.00
3,Adição_AP_1.json,0,3,0.625323,0.153082,774,1006,-1.36,1.00
4,Adição_AP_1.json,0,4,0.543928,0.154076,774,1006,-1.36,1.00
5,Adição_AP_1.json,0,5,0.525840,0.155070,774,1006,-1.36,1.00
6,Adição_AP_1.json,0,6,0.510336,0.157058,774,1006,-1.36,1.00
7,Adição_AP_1.json,0,7,0.652455,0.168986,774,1006,-0.89,1.00
8,Adição_AP_1.json,0,8,0.493540,0.173956,774,1006,-0.85,1.00
9,Adição_AP_1.json,0,9,0.603359,0.217694,774,1006,-1.26,1.00


In [16]:
df_final_longo.shape

(9400809, 9)

In [17]:
df_final_longo.describe()

,frame,id,x,y,width,height,z,visibility
count,9.400809e+06,9.400809e+06,9.400809e+06,9.400809e+06,9.400809e+06,9.400809e+06,9.400809e+06,9.400809e+06
mean,6.396820e+01,1.600000e+01,5.210498e-01,5.732669e-01,7.068266e+02,1.064067e+03,-4.707378e-01,7.732734e-01
std,4.429878e+01,9.521905e+00,1.861914e-01,3.485699e-01,3.108358e+02,5.179935e+02,7.993255e-01,3.913989e-01
min,0.000000e+00,0.000000e+00,0.000000e+00,6.605691e-02,2.180000e+02,2.420000e+02,-3.400000e+00,0.000000e+00
25%,2.800000e+01,8.000000e+00,3.982558e-01,1.888889e-01,4.560000e+02,6.260000e+02,-1.040000e+00,8.400000e-01
50%,5.700000e+01,1.600000e+01,5.212443e-01,5.747508e-01,6.640000e+02,9.380000e+02,-5.800000e-01,1.000000e+00
75%,9.300000e+01,2.400000e+01,6.463768e-01,9.579439e-01,9.160000e+02,1.440000e+03,1.000000e-02,1.000000e+00
max,3.270000e+02,3.200000e+01,1.000000e+00,1.000000e+00,1.918000e+03,2.128000e+03,2.290000e+00,1.000000e+00


### Estratégia de Seleção de Keypoints Relevantes

Uma análise da documentação do corpus e da visibilidade dos keypoints (`visibility`) revelou que os pontos com IDs de 23 a 32, correspondentes ao quadril e aos membros inferiores (pernas e pés), frequentemente apresentavam baixa visibilidade e não são primariamente utilizados na execução da maioria dos sinais em Libras.

Adicionalmente, os keypoints de 0 a 10, que mapeiam a cabeça e o rosto, embora importantes para expressões faciais, foram removidos nesta etapa para focar o modelo exclusivamente nos padrões de movimento dos braços e mãos, que constituem a informação mais discriminante para a classificação dos sinais deste corpus.

Portanto, a estratégia adotada foi filtrar o dataset para manter apenas os **keypoints de ID 11 a 22**, que representam os ombros, cotovelos, pulsos e dedos principais, otimizando o conjunto de dados para a tarefa de reconhecimento de gestos.

In [18]:
## Verifiquei que
# Número de linhas antes da remoção
linhas_antes = len(df_final_longo)

# Remove keypoints com id de 23 a 32
#Partes do corpo que não são necessárias tronco e pernas
df_final_longo = df_final_longo[~df_final_longo['id'].isin(range(23, 33))]

# Número de linhas depois da remoção
linhas_depois = len(df_final_longo)

# Quantidade de linhas removidas
linhas_removidas = linhas_antes - linhas_depois

print(f"Linhas removidas: {linhas_removidas}")


Linhas removidas: 2848730


In [19]:
df_final_longo 

,file_name,frame,id,x,y,width,height,z,visibility
0,Adição_AP_1.json,0,0,0.565891,0.182903,774,1006,-1.44,1.00
1,Adição_AP_1.json,0,1,0.596899,0.153082,774,1006,-1.36,1.00
2,Adição_AP_1.json,0,2,0.612403,0.153082,774,1006,-1.36,1.00
3,Adição_AP_1.json,0,3,0.625323,0.153082,774,1006,-1.36,1.00
4,Adição_AP_1.json,0,4,0.543928,0.154076,774,1006,-1.36,1.00
...,...,...,...,...,...,...,...,...,...
9400794,Ângulo_TS_9.json,140,18,0.071782,0.926045,404,622,-1.24,0.97
9400795,Ângulo_TS_9.json,140,19,0.851485,0.916399,404,622,-0.95,0.98
9400796,Ângulo_TS_9.json,140,20,0.121287,0.929260,404,622,-1.41,0.97
9400797,Ângulo_TS_9.json,140,21,0.836634,0.893891,404,622,-0.72,0.98


In [20]:
## Remoção das partes da cabeça junto com as colunas visibility,
## widh, height e z, do df_final_longo

# Número de linhas antes da remoção
linhas_antes = len(df_final_longo)

# Remove keypoints do rosto (id 0 a 10)
#Parte da cabeça que não são necessárias
df_final_longo = df_final_longo[~df_final_longo['id'].isin(range(0, 11))]

# Número de linhas depois da remoção
linhas_depois = len(df_final_longo)

# Quantidade de linhas removidas
linhas_removidas = linhas_antes - linhas_depois

print(f"Linhas removidas: {linhas_removidas}")

# Remove as colunas se existirem
colunas_para_remover = ['visibility', 'width', 'height', 'z']
df_final_longo = df_final_longo.drop(columns=[col for col in colunas_para_remover if col in df_final_longo.columns])
print("Colunas 'visibility', 'width' e 'height' removidas (se existiam)")



Linhas removidas: 3133603
Colunas 'visibility', 'width' e 'height' removidas (se existiam)


### Simplificação do DataFrame Longo

Após a etapa de normalização, onde as colunas `width` e `height` já cumpriram seu propósito, o DataFrame longo foi simplificado. As colunas `z` (profundidade) e `visibility`, que poderiam adicionar ruído ou complexidade desnecessária nesta fase da modelagem, foram removidas. As colunas `width` e `height` também foram descartadas, pois a informação de escala já estava contida nas novas colunas normalizadas.

O DataFrame resultante (`df_final_longo`) foi então composto apenas pelas colunas essenciais para a próxima fase de engenharia de features: `file_name`, `frame`, `id`, e as coordenadas normalizadas `x` e `y`.

In [21]:
df_final_longo.head(13)

,file_name,frame,id,x,y
11,Adição_AP_1.json,0,11,0.808786,0.387674
12,Adição_AP_1.json,0,12,0.365633,0.394632
13,Adição_AP_1.json,0,13,0.904393,0.638171
14,Adição_AP_1.json,0,14,0.289406,0.655070
15,Adição_AP_1.json,0,15,0.723514,0.793241
16,Adição_AP_1.json,0,16,0.497416,0.805169
17,Adição_AP_1.json,0,17,0.678295,0.859841
18,Adição_AP_1.json,0,18,0.577519,0.861829
19,Adição_AP_1.json,0,19,0.657623,0.827038
20,Adição_AP_1.json,0,20,0.605943,0.819085


In [22]:
# Verificando se há valores NaN depois da normalização
print("Total de NaN:", df_final_longo.isna().sum().sum())

Total de NaN: 0


In [23]:
# Salvar para usar depois sem precisar refazer todo o processo
df_backup = df_final_longo.copy()


### Função para Engenharia de Features Temporais

Para transformar a sequência de movimentos de cada vídeo em um vetor de características de tamanho fixo, foi criada a função `extrair_features_temporais`. O objetivo desta função é receber todos os dados de um único vídeo (no formato longo) e aplicar a seguinte lógica:

1.  **Segmentação Temporal:** A sequência de frames é dividida em 10 partes proporcionais.
2.  **Extração de Estatísticas:** Para cada uma das 10 partes, a função calcula a `média`, `desvio padrão` e `variância` das coordenadas `x` e `y` de cada keypoint individualmente.

O resultado é uma única linha (uma `pd.Series`) que contém 720 features, representando um resumo numérico da trajetória e da dinâmica do sinal ao longo do tempo.

In [24]:
# Função para extrair features temporais
# Divide em grupos(partes) de 10 e calcula mean, std e var para x e y
import pandas as pd

def extrair_features_temporais(video_df, num_partes=10):
    """
    Extrai características temporais de um vídeo (dataframe de keypoints por frame).
    Divide o vídeo em partes e calcula mean, std e var para x e y,
    deixando cada métrica de x e y lado a lado.
    """

    # Pivotar para (frame, keypoint) com x e y
    pivot_video = video_df.pivot(index='frame', columns='id', values=['x','y'])
    pivot_video.columns = [f"kp{kp}_{coord}" for coord, kp in pivot_video.columns]
    pivot_video = pivot_video.sort_index()

    num_frames = len(pivot_video)
    if num_frames == 0:
        return None

    frames_por_parte = max(1, num_frames // num_partes)
    features_finais = []

    for i in range(num_partes):
        start = i * frames_por_parte
        end = (i + 1) * frames_por_parte if i < num_partes - 1 else num_frames
        parte = pivot_video.iloc[start:end]
        if parte.empty:
            continue

        # Calcular estatísticas
        mean_features = parte.mean().to_dict()
        std_features  = parte.std().to_dict()
        var_features  = parte.var().to_dict()

        # Reorganizar colunas para cada keypoint: (x_métrica, y_métrica)
        ordered_features = {}
        for col in sorted(parte.columns, key=lambda x: (int(x.split("_")[0][2:]), x)):
            kp, coord = col.split("_")
            if coord == "x":
                ordered_features[f"{kp}_x_p{i+1}_mean"] = mean_features[col]
                ordered_features[f"{kp}_y_p{i+1}_mean"] = mean_features[f"{kp}_y"]
                ordered_features[f"{kp}_x_p{i+1}_std"]  = std_features[col]
                ordered_features[f"{kp}_y_p{i+1}_std"]  = std_features[f"{kp}_y"]
                ordered_features[f"{kp}_x_p{i+1}_var"]  = var_features[col]
                ordered_features[f"{kp}_y_p{i+1}_var"]  = var_features[f"{kp}_y"]

        features_finais.append(pd.Series(ordered_features))

    if not features_finais:
        return None

    return pd.concat(features_finais)


In [25]:
## Executando a função 

#  Lista para armazenar features de todos os vídeos
features_videos = []

#  Agrupar por 'file_name' (cada vídeo)
for file_name, video_df in df_final_longo.groupby('file_name'):
    features = extrair_features_temporais(video_df, num_partes=10)
    if features is not None:
        # Adiciona o nome do arquivo como coluna
        features['file_name'] = file_name
        features_videos.append(features)

#  DataFrame final (formato wide, em partes)
df_features_temporais = pd.DataFrame(features_videos)

### Aplicação e Verificação das Features

A função `extrair_features_temporais` foi então aplicada a cada vídeo (agrupado por `file_name`) do nosso conjunto de dados. O resultado foi consolidado em um novo DataFrame chamado `df_features_temporais`.

Uma verificação inicial com o método `.head()` confirmou que a estrutura estava correta: cada linha representava um vídeo (`file_name`) e as colunas seguiam o formato desejado (`kp11_x_p1_mean`, `kp11_y_p1_mean`, `kp11_x_p1_std`, etc.), com as métricas de `x` e `y` devidamente intercaladas para cada keypoint em cada parte. 

Leitura para `"kp11_x_p1_mean"`: média de x, do keypoint 11, na parte 1 (ou grupo 1). 

In [26]:
#
df_features_temporais.head()

,kp11_x_p1_mean,kp11_y_p1_mean,kp11_x_p1_std,kp11_y_p1_std,kp11_x_p1_var,kp11_y_p1_var,kp12_x_p1_mean,kp12_y_p1_mean,kp12_x_p1_std,kp12_y_p1_std,...,kp21_y_p10_std,kp21_x_p10_var,kp21_y_p10_var,kp22_x_p10_mean,kp22_y_p10_mean,kp22_x_p10_std,kp22_y_p10_std,kp22_x_p10_var,kp22_y_p10_var,file_name
0,0.806571,0.382633,7.897285e-04,0.001848,6.236711e-07,3.414942e-06,0.363972,0.392289,1.556056e-03,0.001002,...,0.000391,5.056807e-06,1.525754e-07,0.569767,0.812069,0.000000,0.001137,0.000000e+00,1.293258e-06,Adição_AP_1.json
1,0.813008,0.380800,1.564635e-03,0.000979,2.448082e-06,9.589543e-07,0.353659,0.377137,5.777784e-17,0.000373,...,0.001009,9.878227e-07,1.018723e-06,0.558337,0.804198,0.001888,0.000888,3.564751e-06,7.885031e-07,Adição_AP_10.json
2,0.813158,0.382236,1.155557e-16,0.000815,1.335311e-32,6.640080e-07,0.367105,0.387302,5.777784e-17,0.001113,...,0.000805,3.967567e-07,6.474078e-07,0.577878,0.798840,0.000987,0.000812,9.738573e-07,6.598579e-07,Adição_AP_2.json
3,0.818898,0.381500,1.152133e-16,0.000760,1.327410e-32,5.769231e-07,0.375141,0.388214,4.765571e-04,0.000426,...,0.000653,5.554364e-07,4.264069e-07,0.585839,0.801182,0.000660,0.000795,4.361481e-07,6.320346e-07,Adição_AP_3.json
4,0.815344,0.382547,9.941813e-04,0.001492,9.883965e-07,2.225748e-06,0.365183,0.390515,1.155557e-16,0.000492,...,0.001024,4.903074e-06,1.048736e-06,0.585515,0.808243,0.000756,0.000925,5.710735e-07,8.550507e-07,Adição_AP_4.json


In [ ]:
# Salvar o DataFrame final(versao wide, em partes) em CSV
# para caso eu queria testar outras estrategias, tenho o csv longo
df_features_temporais.to_csv("df_features_temporais.csv", index=False, encoding="utf-8-sig")

In [28]:
# Verificando se o as linhas dessa versão wide(larga) do dataframe
# bate com as linhas de sinais.csv
df_features_temporais.shape

(2501, 721)

In [29]:
# Coloca a coluna file_name no início
cols = ['file_name'] + [c for c in df_features_temporais.columns if c != 'file_name']
df_features_temporais = df_features_temporais[cols]


In [30]:
# Verificando se a coluna veio pro inicio
df_features_temporais.head()

,file_name,kp11_x_p1_mean,kp11_y_p1_mean,kp11_x_p1_std,kp11_y_p1_std,kp11_x_p1_var,kp11_y_p1_var,kp12_x_p1_mean,kp12_y_p1_mean,kp12_x_p1_std,...,kp21_x_p10_std,kp21_y_p10_std,kp21_x_p10_var,kp21_y_p10_var,kp22_x_p10_mean,kp22_y_p10_mean,kp22_x_p10_std,kp22_y_p10_std,kp22_x_p10_var,kp22_y_p10_var
0,Adição_AP_1.json,0.806571,0.382633,7.897285e-04,0.001848,6.236711e-07,3.414942e-06,0.363972,0.392289,1.556056e-03,...,0.002249,0.000391,5.056807e-06,1.525754e-07,0.569767,0.812069,0.000000,0.001137,0.000000e+00,1.293258e-06
1,Adição_AP_10.json,0.813008,0.380800,1.564635e-03,0.000979,2.448082e-06,9.589543e-07,0.353659,0.377137,5.777784e-17,...,0.000994,0.001009,9.878227e-07,1.018723e-06,0.558337,0.804198,0.001888,0.000888,3.564751e-06,7.885031e-07
2,Adição_AP_2.json,0.813158,0.382236,1.155557e-16,0.000815,1.335311e-32,6.640080e-07,0.367105,0.387302,5.777784e-17,...,0.000630,0.000805,3.967567e-07,6.474078e-07,0.577878,0.798840,0.000987,0.000812,9.738573e-07,6.598579e-07
3,Adição_AP_3.json,0.818898,0.381500,1.152133e-16,0.000760,1.327410e-32,5.769231e-07,0.375141,0.388214,4.765571e-04,...,0.000745,0.000653,5.554364e-07,4.264069e-07,0.585839,0.801182,0.000660,0.000795,4.361481e-07,6.320346e-07
4,Adição_AP_4.json,0.815344,0.382547,9.941813e-04,0.001492,9.883965e-07,2.225748e-06,0.365183,0.390515,1.155557e-16,...,0.002214,0.001024,4.903074e-06,1.048736e-06,0.585515,0.808243,0.000756,0.000925,5.710735e-07,8.550507e-07


In [31]:
# Verificando se há valores NaN
print("Total de NaN:", df_features_temporais.isna().sum().sum())

Total de NaN: 0


In [32]:
# Cria uma cópia do df final com todos os keypoints inidivuais
df_keypoints = df_final_longo.copy()

# Salva em um arquivo para usar depois
df_keypoints.to_csv("df_keypoints.csv", index=False)

### Construção da Base de Dados Final para Modelagem

Para finalizar a preparação, foram executados os seguintes passos:

* **Junção Final (Merge):** O `df_features_temporais` (contendo as 720 features) foi mesclado com o DataFrame `data` (criado a partir de `sinais.csv`), utilizando `file_name` como chave, resultando no DataFrame final `data_merged1`.

* **Normalização de Texto:** Para garantir a consistência dos dados categóricos e evitar possíveis erros em bibliotecas de Machine Learning, foi aplicada a função `unidecode` nas colunas `sinal` e `interprete`, removendo acentos e caracteres especiais.

* **Limpeza de Metadados:** Por fim, as colunas de metadados restantes (`width`, `height`, `duration_sec`, `num_frames`) foram removidas do `data_merged1`. A remoção dessas colunas é uma etapa crucial, pois elas não representam características do movimento em si. Mantê-las poderia levar o modelo a aprender "atalhos" indesejados (por exemplo, associar a duração de um vídeo a um sinal específico), em vez de aprender os padrões de movimento, o que comprometeria sua capacidade de generalização.

Com isso, a base `data_merged1` ficou pronta para a fase de treinamento e avaliação dos modelos de classificação.

In [33]:
# Merge dos dados do wide com o dataframe original
data_merged1 = data.merge(
    df_features_temporais,          # dataframe com wide
    on='file_name',           # coluna que será chave
    how='left'                # mantém todas as linhas do data
)

# Verificar como ficou
# Dataframe final com todas as colunas para treinar o modelo
data_merged1.head()


,file_name,width,height,duration_sec,num_frames,sinal,interprete,kp11_x_p1_mean,kp11_y_p1_mean,kp11_x_p1_std,...,kp21_x_p10_std,kp21_y_p10_std,kp21_x_p10_var,kp21_y_p10_var,kp22_x_p10_mean,kp22_y_p10_mean,kp22_x_p10_std,kp22_y_p10_std,kp22_x_p10_var,kp22_y_p10_var
0,Adição_AP_10.json,738,1008,4.533333,136,Adição,Alexson,0.813008,0.380800,1.564635e-03,...,0.000994,0.001009,9.878227e-07,1.018723e-06,0.558337,0.804198,0.001888,0.000888,3.564751e-06,7.885031e-07
1,Adição_AP_1.json,774,1006,4.766667,143,Adição,Alexson,0.806571,0.382633,7.897285e-04,...,0.002249,0.000391,5.056807e-06,1.525754e-07,0.569767,0.812069,0.000000,0.001137,0.000000e+00,1.293258e-06
2,Adição_AP_2.json,760,1002,4.433333,133,Adição,Alexson,0.813158,0.382236,1.155557e-16,...,0.000630,0.000805,3.967567e-07,6.474078e-07,0.577878,0.798840,0.000987,0.000812,9.738573e-07,6.598579e-07
3,Adição_AP_3.json,762,1000,4.933333,148,Adição,Alexson,0.818898,0.381500,1.152133e-16,...,0.000745,0.000653,5.554364e-07,4.264069e-07,0.585839,0.801182,0.000660,0.000795,4.361481e-07,6.320346e-07
4,Adição_AP_4.json,764,1004,4.600000,138,Adição,Alexson,0.815344,0.382547,9.941813e-04,...,0.002214,0.001024,4.903074e-06,1.048736e-06,0.585515,0.808243,0.000756,0.000925,5.710735e-07,8.550507e-07


In [34]:
# Lista de colunas que queremos remover do DataFrame
cols_to_remove = ['width', 'height', 'duration_sec', 'num_frames']

# Remove as colunas especificadas se existirem no DataFrame data_merged1
data_merged1 = data_merged1.drop(columns=[col for col in cols_to_remove if col in data_merged1.columns])

In [35]:
import unidecode

# Lista das colunas de texto que queremos limpar
colunas_texto = ['sinal', 'interprete']

# Remove acentos
for col in colunas_texto:
    data_merged1[col] = data_merged1[col].astype(str).apply(unidecode.unidecode)


In [36]:
data_merged1.head(50)

,file_name,sinal,interprete,kp11_x_p1_mean,kp11_y_p1_mean,kp11_x_p1_std,kp11_y_p1_std,kp11_x_p1_var,kp11_y_p1_var,kp12_x_p1_mean,...,kp21_x_p10_std,kp21_y_p10_std,kp21_x_p10_var,kp21_y_p10_var,kp22_x_p10_mean,kp22_y_p10_mean,kp22_x_p10_std,kp22_y_p10_std,kp22_x_p10_var,kp22_y_p10_var
0,Adição_AP_10.json,Adicao,Alexson,0.813008,0.380800,1.564635e-03,0.000979,2.448082e-06,9.589543e-07,0.353659,...,0.000994,1.009318e-03,9.878227e-07,1.018723e-06,0.558337,0.804198,0.001888,0.000888,3.564751e-06,7.885031e-07
1,Adição_AP_1.json,Adicao,Alexson,0.806571,0.382633,7.897285e-04,0.001848,6.236711e-07,3.414942e-06,0.363972,...,0.002249,3.906090e-04,5.056807e-06,1.525754e-07,0.569767,0.812069,0.000000,0.001137,0.000000e+00,1.293258e-06
2,Adição_AP_2.json,Adicao,Alexson,0.813158,0.382236,1.155557e-16,0.000815,1.335311e-32,6.640080e-07,0.367105,...,0.000630,8.046165e-04,3.967567e-07,6.474078e-07,0.577878,0.798840,0.000987,0.000812,9.738573e-07,6.598579e-07
3,Adição_AP_3.json,Adicao,Alexson,0.818898,0.381500,1.152133e-16,0.000760,1.327410e-32,5.769231e-07,0.375141,...,0.000745,6.529984e-04,5.554364e-07,4.264069e-07,0.585839,0.801182,0.000660,0.000795,4.361481e-07,6.320346e-07
4,Adição_AP_4.json,Adicao,Alexson,0.815344,0.382547,9.941813e-04,0.001492,9.883965e-07,2.225748e-06,0.365183,...,0.002214,1.024078e-03,4.903074e-06,1.048736e-06,0.585515,0.808243,0.000756,0.000925,5.710735e-07,8.550507e-07
5,Adição_AP_5.json,Adicao,Alexson,0.814759,0.380854,1.295402e-03,0.000959,1.678066e-06,9.193957e-07,0.364829,...,0.000681,3.624117e-04,4.636761e-07,1.313422e-07,0.576209,0.810807,0.002024,0.000513,4.097383e-06,2.626845e-07
6,Adição_AP_6.json,Adicao,Alexson,0.823972,0.386917,4.617661e-04,0.000609,2.132279e-07,3.705056e-07,0.369729,...,0.002262,1.382560e-03,5.114488e-06,1.911472e-06,0.601348,0.805630,0.002140,0.002316,4.581418e-06,5.363911e-06
7,Adição_AP_7.json,Adicao,Alexson,0.812916,0.377684,6.652236e-04,0.000732,4.425224e-07,5.351053e-07,0.375648,...,0.001358,1.008490e-03,1.844961e-06,1.017051e-06,0.604746,0.791629,0.003399,0.001177,1.155280e-05,1.384320e-06
8,Adição_AP_8.json,Adicao,Alexson,0.819306,0.384747,9.952570e-04,0.001432,9.905364e-07,2.049505e-06,0.375600,...,0.000864,8.966675e-04,7.473383e-07,8.040126e-07,0.586779,0.797587,0.002776,0.000805,7.707949e-06,6.483972e-07
9,Adição_AP_9.json,Adicao,Alexson,0.823301,0.389356,9.226517e-04,0.000619,8.512862e-07,3.835446e-07,0.374886,...,0.005268,2.713347e-03,2.774705e-05,7.362252e-06,0.604451,0.796311,0.006030,0.004112,3.636171e-05,1.690804e-05


In [37]:
# Verificando o shape
data_merged1.shape

(2501, 723)

In [38]:
# Verificando se há valores NaN no nosso dataframe
print("Total de NaN:", data_merged1.isna().sum().sum())

Total de NaN: 0


In [39]:
data_merged1.to_csv("data_merged1.csv", index=False, encoding="utf-8-sig")